![Screenshot 2024-11-25 at 7 12 58 PM](https://github.com/user-attachments/assets/48dd9de1-b4d2-4318-8f52-85ec209d8ebc)

# Install Necessary Libraries

In [1]:
! pip install pygithub langchain langchain-community openai tiktoken pinecone-client langchain_pinecone sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 74.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3

In [2]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from langchain_pinecone import PineconeVectorStore
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from google.colab import userdata
from pinecone import Pinecone
import os
import tempfile
from github import Github, Repository
from git import Repo
from openai import OpenAI
from pathlib import Path
from langchain.schema import Document
from pinecone import Pinecone

# Clone a GitHub Repo locally

In [3]:
github_repo="https://github.com/CoderAgent/SecureAgent"
github_repo.split("/")[-1]

'SecureAgent'

In [5]:
def clone_repository(repo_url):
    """Clones a GitHub repository to a temporary directory.

    Args:
        repo_url: The URL of the GitHub repository.

    Returns:
        The path to the cloned repository.
    """

    repo_name = github_repo.split("/")[-1]
    repo_path = f"/content/{repo_name}"
    Repo.clone_from(repo_url, str(repo_path))
    return repo_path

In [6]:
path = clone_repository(github_repo)

# Define which types of files to parse and which files / folders to ignore

In [7]:
SUPPORTED_EXTENSIONS = {'.py', '.js', '.tsx', '.jsx', '.ipynb', '.java',
                         '.cpp', '.ts', '.go', '.rs', '.vue', '.swift', '.c', '.h'}

IGNORED_DIRS = {'node_modules', 'venv', 'env', 'dist', 'build', '.git',
                '__pycache__', '.next', '.vscode', 'vendor'}

In [12]:
def get_file_content(file_path, repo_path):
    """
    Get content of a single file.

    Args:
        file_path (str): Path to the file

    Returns:
        Optional[Dict[str, str]]: Dictionary with file name and content
    """
    try:
      with open(file_path,"r",encoding="utf-8") as f:
        content = f.read()

      rel_path = os.path.relpath(file_path, repo_path)

      return {
          "name": rel_path,
          "content": content
      }


    except Exception as e:
      print(f"Error processing file {file_path}: {str(e)}")
      return None


def get_main_files_content(repo_path: str):
    """
    Get content of supported code files from the local repository.

    Args:
        repo_path: Path to the local repository

    Returns:
        List of dictionaries containing file names and contents
    """
    files_content = []

    try:
      for root, _, files in os.walk(repo_path):
        for file in files:
          if any(ignored_dir in root for ignored_dir in IGNORED_DIRS):
            continue
          for file in files:
            file_path = os.path.join(root, file)
            if os.path.splitext(file_path)[1] in SUPPORTED_EXTENSIONS:
              file_content = get_file_content(file_path, repo_path)
              if file_content:
                files_content.append(file_content)
    except Exception as e:
      print(e)

    return files_content

In [13]:
files_content = get_main_files_content(path)

# Embeddings

In [10]:
def get_huggingface_embeddings(text, model_name="sentence-transformers/all-mpnet-base-v2"):
    model = SentenceTransformer(model_name)
    return model.encode(text)

In [11]:
text = "I like coding"
embeddings = get_huggingface_embeddings(text)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [14]:
embeddings

array([-8.99214391e-03,  7.10570812e-02, -4.78132404e-02,  2.93951924e-03,
        1.87183879e-02,  6.25578165e-02, -7.40241334e-02, -1.79869793e-02,
       -2.33030953e-02,  2.52969563e-02,  1.87584516e-02,  3.26438658e-02,
        2.42187642e-02,  1.06377289e-01,  1.96239557e-02,  4.58664587e-03,
       -2.44689845e-02, -2.76926570e-02, -3.92995449e-03, -1.75862219e-02,
        7.31050968e-03,  3.00472043e-02, -3.48030999e-02, -1.51053397e-02,
        4.95744497e-03, -1.16299819e-02,  2.20015477e-02, -6.58035874e-02,
       -2.77131312e-02,  1.00825511e-01,  3.28472641e-04, -9.48085822e-03,
       -2.19271723e-02, -4.70387796e-03,  1.50509811e-06,  1.31359547e-02,
       -1.65854990e-02,  1.93589348e-02, -2.62506306e-02,  1.61743432e-04,
       -1.85249373e-02, -1.79329310e-02, -2.03854069e-02,  2.96985377e-02,
       -5.77551797e-02,  1.27474498e-02,  8.77798274e-02, -6.04865141e-02,
        9.92059615e-03, -2.96356127e-04,  1.08571432e-03, -1.75144020e-02,
       -3.35006081e-02, -

# Setting up Pinecone
**1. Create an account on [Pinecone.io](https://app.pinecone.io/)**

**2. Create a new index called "codebase-rag" and set the dimensions to 768. Leave the rest of the settings as they are.**

![Screenshot 2024-11-24 at 10 58 50 PM](https://github.com/user-attachments/assets/f5fda046-4087-432a-a8c2-86e061005238)



**3. Create an API Key for Pinecone**

![Screenshot 2024-11-24 at 10 44 37 PM](https://github.com/user-attachments/assets/e7feacc6-2bd1-472a-82e5-659f65624a88)


**4. Store your Pinecone API Key within Google Colab's secrets section, and then enable access to it (see the blue checkmark)**

![Screenshot 2024-11-24 at 10 45 25 PM](https://github.com/user-attachments/assets/eaf73083-0b5f-4d17-9e0c-eab84f91b0bc)



In [15]:
# Set the PINECONE_API_KEY as an environment variable
pinecone_api_key = userdata.get("PINECONE_API_KEY")
os.environ['PINECONE_API_KEY'] = pinecone_api_key

# Initialize Pinecone
pc = Pinecone(api_key=userdata.get("PINECONE_API_KEY"),)

# Connect to your Pinecone index
pinecone_index = pc.Index("codebase-rag")

In [16]:
vectorstore = PineconeVectorStore(index_name="codebase-rag", embedding=HuggingFaceEmbeddings())

<ipython-input-16-5982ffb8f713>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  vectorstore = PineconeVectorStore(index_name="codebase-rag", embedding=HuggingFaceEmbeddings())
<ipython-input-16-5982ffb8f713>:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  vectorstore = PineconeVectorStore(index_name="codebase-rag", embedding=HuggingFaceEmbeddings())


In [19]:
# Insert the codebase embeddings into Pinecone
documents= []
for file in files_content:
  doc = Document(
      page_content=f"{file['name']}\n\n{file['content']}",
      metadata={
          "source": file["name"]
      }
  )
  documents.append(doc)

vectorstore = PineconeVectorStore.from_documents(
    documents=documents,
    embedding=HuggingFaceEmbeddings(),
    index_name="codebase-rag",
    namespace="https://github.com/CoderAgent/SecureAgent"
)





<ipython-input-19-c9ebd7a63d6e>:14: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedding=HuggingFaceEmbeddings(),


In [18]:
documents

[Document(metadata={'source': 'src/review-agent.ts'}, page_content='src/review-agent.ts\n\nimport { Octokit } from "@octokit/rest";\nimport { WebhookEventMap } from "@octokit/webhooks-definitions/schema";\nimport { ChatCompletionMessageParam } from "groq-sdk/resources/chat/completions";\nimport * as xml2js from "xml2js";\nimport type {\n  BranchDetails,\n  BuilderResponse,\n  Builders,\n  CodeSuggestion,\n  PRFile,\n  PRSuggestion,\n} from "./constants";\nimport { PRSuggestionImpl } from "./data/PRSuggestionImpl";\nimport { generateChatCompletion } from "./llms/chat";\nimport {\n  PR_SUGGESTION_TEMPLATE,\n  buildPatchPrompt,\n  constructPrompt,\n  getReviewPrompt,\n  getTokenLength,\n  getXMLReviewPrompt,\n  isConversationWithinLimit,\n} from "./prompts";\nimport {\n  INLINE_FIX_FUNCTION,\n  getInlineFixPrompt,\n} from "./prompts/inline-prompt";\nimport { getGitFile } from "./reviews";\n\nexport const reviewDiff = async (messages: ChatCompletionMessageParam[]) => {\n  const message = a

# Perform RAG

1. Get your OpenRouter API Key [here](https://openrouter.ai/settings/keys)

2. Paste your OpenRouter Key into your Google Colab secrets, and make sure to enable permissions for it

![Image](https://github.com/user-attachments/assets/bd64c5aa-952e-4a1e-9ac0-01d8fe93aaa1)


In [20]:
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=userdata.get("OPENROUTER_API_KEY")
)

In [21]:
query = "How is the python parser used"

In [22]:
raw_query_embedding = get_huggingface_embeddings(query)

In [23]:
raw_query_embedding

array([ 5.43670058e-02, -4.33224887e-02, -5.88456355e-03,  1.38295572e-02,
       -2.87646241e-02,  5.98886721e-02,  7.98857212e-03, -1.36434855e-02,
       -1.87077727e-02, -7.81196654e-02,  3.06505524e-02,  1.50619783e-02,
        4.69016470e-02,  5.22097647e-02, -1.07367476e-02, -4.81214002e-02,
        2.01661531e-02, -4.90508974e-03,  3.75727862e-02,  4.20053788e-02,
       -3.41497101e-02,  3.70276384e-02, -1.19858552e-02,  4.01923694e-02,
       -9.61441360e-03,  1.96691491e-02, -3.34812654e-03,  5.70160151e-03,
       -3.80436443e-02, -7.12570921e-02, -3.52873243e-02, -2.67340150e-02,
        2.99257021e-02, -8.72928556e-03,  1.10090707e-06, -1.98547053e-03,
       -6.06353655e-02,  2.00967938e-02, -3.48655619e-02,  2.96150874e-02,
        7.68796727e-02, -6.18706346e-02,  3.88548449e-02, -3.62081314e-03,
       -4.48348261e-02, -2.81078946e-02,  4.92585897e-02, -4.28294800e-02,
        7.51839057e-02,  6.19241744e-02, -5.07643819e-03, -4.48793285e-02,
       -1.01818154e-02,  

In [25]:
top_matches = pinecone_index.query(
    top_k=5,
    vector=raw_query_embedding.tolist(),
    include_metadata=True,
    namespace = "https://github.com/CoderAgent/SecureAgent"
)

In [26]:
top_matches

{'matches': [{'id': '4866234c-d364-4cd6-b242-69e4c91f23c4',
              'metadata': {'source': 'src/context/language/python-parser.ts',
                           'text': 'src/context/language/python-parser.ts\n'
                                   '\n'
                                   'import { AbstractParser, EnclosingContext '
                                   '} from "../../constants";\n'
                                   'export class PythonParser implements '
                                   'AbstractParser {\n'
                                   '  findEnclosingContext(\n'
                                   '    file: string,\n'
                                   '    lineStart: number,\n'
                                   '    lineEnd: number\n'
                                   '  ): EnclosingContext {\n'
                                   '    // TODO: Implement this method for '
                                   'Python\n'
                                   '    re

In [30]:
context = [item.metadata["text"] for item in top_matches['matches'] if "text" in item.metadata]


In [31]:
augmented_query = "<CONTEXT>\n" + "\n\n-------\n\n".join(context) + "\n-------\n</CONTEXT>\n\n\n\nMY QUESTION:\n" + query

In [34]:
print(augmented_query)

<CONTEXT>
src/context/language/python-parser.ts

import { AbstractParser, EnclosingContext } from "../../constants";
export class PythonParser implements AbstractParser {
  findEnclosingContext(
    file: string,
    lineStart: number,
    lineEnd: number
  ): EnclosingContext {
    // TODO: Implement this method for Python
    return null;
  }
  dryRun(file: string): { valid: boolean; error: string } {
    // TODO: Implement this method for Python
    return { valid: false, error: "Not implemented yet" };
  }
}


-------

src/context/language/python-parser.ts

import { AbstractParser, EnclosingContext } from "../../constants";
export class PythonParser implements AbstractParser {
  findEnclosingContext(
    file: string,
    lineStart: number,
    lineEnd: number
  ): EnclosingContext {
    // TODO: Implement this method for Python
    return null;
  }
  dryRun(file: string): { valid: boolean; error: string } {
    // TODO: Implement this method for Python
    return { valid: false, e

In [32]:
system_prompt="""You are a Senior software Engineer, who is an expert in Typescript."
Answer the question I have about the codebase based on the context provided. Always consider all of the context provided
"""

llm_response = client.chat.completions.create(
    model="deepseek/deepseek-r1:free",
    messages=[
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": augmented_query
        }
    ]
)
response=llm_response.choices[0].message.content

In [36]:
print(response)

The Python parser in the provided codebase is **currently not implemented and is a placeholder**. Here's the breakdown of its usage and status:

1. **Structure**:  
   - Implements the `AbstractParser` interface (from `../../constants`), requiring `findEnclosingContext` and `dryRun` methods.
   - Located at `src/context/language/python-parser.ts`.

2. **Current State**:  
   - Both critical methods **return hardcoded values** (`null`/`false`) with TODOs.
   - Contrasts with `JavascriptParser` which fully implements AST parsing using Babel.

3. **Intended Purpose** (based on JS implementation):  
   - **`findEnclosingContext`**  
     Would analyze Python AST to find the largest surrounding context (like classes/functions) for given line numbers.
   - **`dryRun`**  
     Would validate Python syntax by attempting to parse the file.

4. **Actual Usage**:  
   Any code calling `PythonParser` would receive:
   - `null` for context queries
   - `{valid: false}` for syntax checks
   Until th